In [ ]:
import mlflow
import datetime
Current_Date = datetime.datetime.today()

mlflow.set_experiment('ML_Approach_ABTEST-' + str(Current_Date))

In [ ]:
# feature_cols = ["experiment", "hour", "date", 'device_make', "platform_os",  "browser"]
feature_cols = ["experiment", "hour", "date", 'device_make', 'browser']

X = browser_df[feature_cols]
y = browser_df[['aware']]

test_size = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:

decisionTreesModel = DecisionTreesModel(X_train, X_test,  y_train, y_test)

folds = 5
clf, acc_arr, loss_arr = decisionTreesModel.train(folds)

write_model('browser_decision_tree_model', clf)

In [ ]:

test_acc, loss, report, confusion_matrix = decisionTreesModel.test()
print(f"Loss on test data is: {loss:.3f}")
print()


In [ ]:
decisionTreesModel.get_feature_importance()

In [ ]:

test_acc, loss, report, confusion_matrix = decisionTreesModel.test()
print(f"Loss on test data is: {loss:.3f}")
print()


In [ ]:
decisionTreesModel.get_feature_importance()

In [ ]:
dot_data = StringIO()

leaves_parallel=False
feature_cols = ["experiment", "hour", "date", 'device_make', 'browser']
out_put_file = "AbTestDecisionTree.dot"

dot_data = export_graphviz(clf, out_file=out_put_file,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['Aware','Not Aware'])

graph = pydotplus.graphviz.graph_from_dot_file(out_put_file)
graph.write_png('AbTestDecisionTree.png')

Image(graph.create_png())


In [ ]:
mlflow.end_run()
with mlflow.start_run(run_name="Browser-DF-DecisionTree"):
    mlflow.log_param('Features', X_train.columns.to_list())
    mlflow.log_param('Target', y_train.columns.to_list())
    mlflow.log_param('Number Of Training Dataset', X_train.shape[0])
    mlflow.log_param('Number Of Test Dataset', X_test.shape[0])
    mlflow.log_param('Fold number', folds)
    
    mlflow.log_metric('Validation Accuray', acc_arr[-1])
    mlflow.log_metric('Validation Loss', loss_arr[-1])
    mlflow.log_metric('Test accuracy', test_acc)
    mlflow.log_metric('Test Loss MSE', loss)
    
#     mlflow.log_figure("../AbTestDecisionTree.png", 'fig.png')

In [ ]:
# feature_cols = ["experiment", "hour", "date", 'device_make', "platform_os",  "browser"]
feature_cols = ["experiment", "hour", "date", 'device_make', 'platform_os']

X = platform_df[feature_cols]
y = platform_df[['aware']]

test_size = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
decisionTreesModel = DecisionTreesModel(X_train, X_test,  y_train, y_test)

folds = 5
clf, acc_arr, loss_arr = decisionTreesModel.train(folds)

write_model('platform_os_decision_tree_model', clf)

In [ ]:
test_acc, loss, report, confusion_matrix = decisionTreesModel.test()
print(f"Loss on test data is: {loss:.3f}")
print()

In [ ]:
decisionTreesModel.get_feature_importance()

In [ ]:
dot_data = StringIO()

leaves_parallel=False
feature_cols = ["experiment", "hour", "date", 'device_make', 'browser']
out_put_file = "AbTestDecisionTree.dot"

dot_data = export_graphviz(clf, out_file=out_put_file,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['Aware','Not Aware'])

graph = pydotplus.graphviz.graph_from_dot_file(out_put_file)
graph.write_png('AbTestDecisionTree.png')
Image(graph.create_png())

In [ ]:
mlflow.end_run()
with mlflow.start_run(run_name="Platform-DF-DecisionTree"):
    mlflow.log_param('Features', X_train.columns.to_list())
    mlflow.log_param('Target', y_train.columns.to_list())
    mlflow.log_param('Number Of Training Dataset', X_train.shape[0])
    mlflow.log_param('Number Of Test Dataset', X_test.shape[0])
    mlflow.log_param('Fold number', folds)
    
    mlflow.log_metric('Validation Accuray', acc_arr[-1])
    mlflow.log_metric('Validation Loss', loss_arr[-1])
    mlflow.log_metric('Test accuracy', test_acc)
    mlflow.log_metric('Test Loss MSE', loss)
    
#     mlflow.log_figure(graph.create_png(), 'fig.png')

In [ ]:
class LogesticRegressionModel:
    
    def __init__(self, X_train, X_test, y_train, y_test, model_name="LR"):
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.model_name = model_name
        
        self.clf = LogisticRegression()
        
    def train(self, folds=1):
        
        kf = KFold(n_splits = folds)
        
        iterator = kf.split(self.X_train)
        
        loss_arr = []
        acc_arr = []
        model_name= self.model_name
        mlflow.end_run()
        for i in range(folds):

            train_index, valid_index = next(iterator)

            X_train, y_train = self.X_train.iloc[train_index], self.y_train.iloc[train_index]
            X_valid, y_valid = self.X_train.iloc[valid_index], self.y_train.iloc[valid_index]

            self.clf = self.clf.fit(X_train, y_train)

            vali_pred = self.clf.predict(X_valid)

            accuracy = self.calculate_score(y_valid, vali_pred)
            loss = loss_function(y_valid, vali_pred)

            self.__printAccuracy(accuracy, i, label="Validation")
            self.__printLoss(loss, i, label="Validation")
            print()

            acc_arr.append(accuracy)
            loss_arr.append(loss)
            
        return self.clf, acc_arr, loss_arr
    
    def test(self):
        y_pred = self.clf.predict(self.X_test)
        
        accuracy = self.calculate_score(self.y_test, y_pred)
        self.__printAccuracy(accuracy, label="Test")
        
        report = self.report(y_pred, self.y_test)
        matrix = self.confusion_matrix(y_pred, self.y_test)
        loss = loss_function(self.y_test, y_pred)
        
        return accuracy, loss, report, matrix 
    
    def __printAccuracy(self, acc, step=1, label=""):
        print(f"step {step}: {label} Accuracy of LogesticRegression is: {acc:.3f}")
    
    def __printLoss(self, loss, step=1, label=""):
        print(f"step {step}: {label} Loss of LogesticRegression is: {loss:.3f}")
    
    def calculate_score(self, pred, actual):
        return metrics.accuracy_score(actual, pred)
    
    def report(self, pred, actual):
        print("Test Metrics")
        print("================")
        print(metrics.classification_report(pred, actual))
        return metrics.classification_report(pred, actual)
    
    def confusion_matrix(self, pred, actual):
        ax=sns.heatmap(pd.DataFrame(metrics.confusion_matrix(pred, actual)))
        plt.title('Confusion matrix')
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        return metrics.confusion_matrix(pred, actual)
    
    def get_p_values(self):
        """ 
        Calcualting p_values for logestic regression.
        code refered from the following link
        https://gist.github.com/rspeare/77061e6e317896be29c6de9a85db301d
        
        """
        denom = (2.0*(1.0+np.cosh(self.clf.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X/denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.clf.coef_[0]/sigma_estimates # z-score 
        p_values = [stat.norm.sf(abs(x)) for x in z_scores] ### two tailed test for p-values
        
        p_df = pd.DataFrame()
        p_df['features'] = self.X_train.columns.to_list()
        p_df['p_values'] = p_values
        
        return p_df
    
    def plot_pvalues(self, p_df):
        
        fig, ax = plt.subplots(figsize=(12,7))

        ax.plot([0.05,0.05], [0.05,5])
        sns.scatterplot(data=p_df, y='features', x='p_values', color="green")
        plt.title("P values of features", size=20)

        plt.xticks(np.arange(0,max(p_df['p_values']) + 0.05, 0.05))

        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)

        plt.show()
        return fig

In [ ]:
feature_cols = ["experiment", "hour", "date", 'device_make', 'browser']

X = browser_df[feature_cols]
y = browser_df[['aware']]

test_size = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
logesticRegressionModel = LogesticRegressionModel(X_train, X_test,  y_train, y_test)

folds = 5
clf2, loss_arr_2, acc_arr_2 = logesticRegressionModel.train(folds)

write_model('browser_Logestic_Reg_model', clf2)


In [ ]:
test_acc2, test_loss2, report2, matrix2  = logesticRegressionModel.test()

In [ ]:

p_values_df = logesticRegressionModel.get_p_values()
p_value_fig = logesticRegressionModel.plot_pvalues(p_values_df)
p_values_df


In [ ]:

mlflow.end_run()
with mlflow.start_run(run_name="Browser-DF-Logestic-Reg"):
    mlflow.log_param('Features', X_train.columns.to_list())
    mlflow.log_param('Target', y_train.columns.to_list())
    mlflow.log_param('Number Of Training Dataset', X_train.shape[0])
    mlflow.log_param('Number Of Test Dataset', X_test.shape[0])
    mlflow.log_param('Fold number', folds)
    
    mlflow.log_metric('Validation Accuray', acc_arr_2[-1])
    mlflow.log_metric('Validation Loss', acc_arr_2[-1])
    mlflow.log_metric('Test accuracy', test_acc2)
    mlflow.log_metric('Test Loss MSE', test_loss2)
    
    mlflow.log_figure(p_value_fig, 'fig.png')


    

In [ ]:
feature_cols = ["experiment", "hour", "date", 'device_make', 'platform_os']

X = platform_df[feature_cols]
y = platform_df[['aware']]

test_size = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
logesticRegressionModel = LogesticRegressionModel(X_train, X_test,  y_train, y_test)

folds = 5
clf2, loss_arr_2, acc_arr_2 = logesticRegressionModel.train(folds)

write_model('platform_os_Logestic_Reg_model', clf2)


In [ ]:
test_acc2, test_loss2, report2, matrix2  = logesticRegressionModel.test()

In [ ]:
p_values_df = logesticRegressionModel.get_p_values()
p_value_fig = logesticRegressionModel.plot_pvalues(p_values_df)
p_values_df

In [ ]:
mlflow.end_run()
with mlflow.start_run(run_name="Platfrom-DF-Logestic-Reg"):
    mlflow.log_param('Features', X_train.columns.to_list())
    mlflow.log_param('Target', y_train.columns.to_list())
    mlflow.log_param('Number Of Training Dataset', X_train.shape[0])
    mlflow.log_param('Number Of Test Dataset', X_test.shape[0])
    mlflow.log_param('Fold number', folds)
    
    mlflow.log_metric('Validation Accuray', acc_arr_2[-1])
    mlflow.log_metric('Validation Loss', acc_arr_2[-1])
    mlflow.log_metric('Test accuracy', test_acc2)
    mlflow.log_metric('Test Loss MSE', test_loss2)
    
    mlflow.log_figure(p_value_fig, 'fig.png')